In [ ]:
from bs4 import BeautifulSoup
import requests
from dotenv import load_dotenv
import os
from google import genai
from IPython.display import Markdown,display

In [ ]:
load_dotenv(override=True)

In [ ]:
api_key=os.getenv('GEMINI_API_KEY')

In [ ]:
def myFunc(url):
    if not url or not url.startswith("http"):
        return {"title": "", "text": "", "links": []}  
    page_text=requests.get(url).text
    soap=BeautifulSoup(page_text,'lxml')
    title=soap.find("span",class_="mw-page-title-main").text
    text=soap.find("div",class_="mw-body-content").text
    links=soap.find_all("a")
    storedLinks=[]
    for link in links:
        href=link.get('href')
        if(href and "/wiki" in href and not href.startswith("/wiki/File:")):
            storedLinks.append("https://en.wikipedia.org/"+ href)
        
        
    

    return {"text":text,"links":storedLinks,"title":title}

In [ ]:
result=myFunc("https://en.wikipedia.org/wiki/Islamabad")

In [ ]:
print(result['links'])

In [ ]:
client = genai.Client()

In [ ]:
response = client.models.generate_content(
    model="gemini-2.5-flash", contents=f"This is the wikipedia page of city: {result['title']} and I have all the urls on the page, i want you to give me strictly only those links that are related in a way as if i was making a travel guide on the city, this is the page text {result['text']} and these are the links {result['links']} the result should be in a json format"
)
print("Compilign..")
relatedLinks=response.text
print(relatedLinks)

In [ ]:
for link in relatedLinks:
    result['text'] += "\n\n" + myFunc(link)['text']

    

In [ ]:
response = client.models.generate_content(
    model="gemini-2.5-flash", contents=f"This is {result['text']} about the city {result['title']}, using all this information make me a beautifully formatted city tour guide"
)
generated_text = response.candidates[0].content.parts[0].text  

display(Markdown(generated_text))

In [ ]:
g